# Strategy Capacity Analysis

Determine strategy capacity by incrementally increasing capital and measuring performance decay due to slippage.

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime
from copy import deepcopy

from data_acquisition import DataAcquisition
from signal_generator import SignalGenerator
from backtester import Backtester

sns.set_style('darkgrid')
%matplotlib inline

## 1. Setup

In [ ]:
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

data_acq = DataAcquisition(config)
signal_gen = SignalGenerator(config)

start_date = datetime.strptime(config['backtest']['start_date'], '%Y-%m-%d')
end_date = datetime.strptime(config['backtest']['end_date'], '%Y-%m-%d')

df = data_acq.prepare_dataset('BTCUSDT', start_date, end_date)
signals = signal_gen.generate_signals(df)

## 2. Run Capacity Analysis

In [ ]:
# Test different capital levels
capital_levels = [100_000, 250_000, 500_000, 1_000_000, 2_500_000, 5_000_000, 10_000_000]

results = []

for capital in capital_levels:
    print(f"Testing capital: ${capital:,}")
    
    # Update config
    config_test = deepcopy(config)
    config_test['backtest']['initial_capital'] = capital
    
    # Run backtest
    backtester = Backtester(config_test)
    backtest_results, trades = backtester.run_backtest(df, signals)
    metrics = backtester.calculate_metrics(backtest_results, trades)
    
    results.append({
        'Capital': capital,
        'Sharpe Ratio': metrics['Sharpe Ratio'],
        'Annual Return (%)': metrics['Annual Return (%)'],
        'Max Drawdown (%)': metrics['Max Drawdown (%)'],
        'Total Trades': metrics['Total Trades']
    })

results_df = pd.DataFrame(results)
print("\nCapacity Analysis Results:")
print(results_df.to_string(index=False))

## 3. Visualize Performance Decay

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sharpe Ratio vs Capital
axes[0, 0].plot(results_df['Capital'] / 1e6, results_df['Sharpe Ratio'], marker='o', linewidth=2)
baseline_sharpe = results_df['Sharpe Ratio'].iloc[0]
axes[0, 0].axhline(y=baseline_sharpe * 0.8, color='red', linestyle='--', 
                   label=f'20% Decay Threshold ({baseline_sharpe * 0.8:.2f})')
axes[0, 0].set_title('Sharpe Ratio vs Capital', fontweight='bold')
axes[0, 0].set_xlabel('Capital ($M)')
axes[0, 0].set_ylabel('Sharpe Ratio')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Annual Return vs Capital
axes[0, 1].plot(results_df['Capital'] / 1e6, results_df['Annual Return (%)'], 
                marker='o', linewidth=2, color='green')
axes[0, 1].set_title('Annual Return vs Capital', fontweight='bold')
axes[0, 1].set_xlabel('Capital ($M)')
axes[0, 1].set_ylabel('Annual Return (%)')
axes[0, 1].grid(True, alpha=0.3)

# Max Drawdown vs Capital
axes[1, 0].plot(results_df['Capital'] / 1e6, results_df['Max Drawdown (%)'], 
                marker='o', linewidth=2, color='red')
axes[1, 0].axhline(y=-10, color='orange', linestyle='--', label='Risk Limit (-10%)')
axes[1, 0].set_title('Max Drawdown vs Capital', fontweight='bold')
axes[1, 0].set_xlabel('Capital ($M)')
axes[1, 0].set_ylabel('Max Drawdown (%)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Sharpe decay percentage
results_df['Sharpe Decay (%)'] = (results_df['Sharpe Ratio'] / baseline_sharpe - 1) * 100
axes[1, 1].plot(results_df['Capital'] / 1e6, results_df['Sharpe Decay (%)'], 
                marker='o', linewidth=2, color='purple')
axes[1, 1].axhline(y=-20, color='red', linestyle='--', label='20% Decay Threshold')
axes[1, 1].set_title('Sharpe Ratio Decay', fontweight='bold')
axes[1, 1].set_xlabel('Capital ($M)')
axes[1, 1].set_ylabel('Decay (%)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Determine Strategy Capacity

In [ ]:
# Find capacity where Sharpe decays by 20%
threshold_sharpe = baseline_sharpe * 0.8
capacity_row = results_df[results_df['Sharpe Ratio'] <= threshold_sharpe]

if len(capacity_row) > 0:
    capacity = capacity_row.iloc[0]['Capital']
    print(f"\n{'='*60}")
    print(f"STRATEGY CAPACITY ESTIMATE")
    print(f"{'='*60}")
    print(f"Baseline Sharpe Ratio: {baseline_sharpe:.2f}")
    print(f"20% Decay Threshold: {threshold_sharpe:.2f}")
    print(f"\nEstimated Capacity: ${capacity:,.0f}")
    print(f"{'='*60}")
else:
    print(f"\nStrategy capacity exceeds ${capital_levels[-1]:,}")
    print(f"Sharpe Ratio remains above {threshold_sharpe:.2f} at maximum tested capital")

## 5. Slippage Impact Analysis

In [ ]:
# Test different slippage factors
slippage_factors = [0.5, 1.0, 2.0, 3.0, 5.0]
capital_test = 1_000_000

slippage_results = []

for factor in slippage_factors:
    print(f"Testing slippage factor: {factor}")
    
    config_test = deepcopy(config)
    config_test['backtest']['initial_capital'] = capital_test
    config_test['backtest']['slippage_factor'] = factor
    
    backtester = Backtester(config_test)
    backtest_results, trades = backtester.run_backtest(df, signals)
    metrics = backtester.calculate_metrics(backtest_results, trades)
    
    slippage_results.append({
        'Slippage Factor': factor,
        'Sharpe Ratio': metrics['Sharpe Ratio'],
        'Annual Return (%)': metrics['Annual Return (%)'],
        'Win Rate (%)': metrics['Win Rate (%)']
    })

slippage_df = pd.DataFrame(slippage_results)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(slippage_df['Slippage Factor'], slippage_df['Sharpe Ratio'], marker='o', linewidth=2)
axes[0].set_title('Sharpe Ratio vs Slippage Factor', fontweight='bold')
axes[0].set_xlabel('Slippage Factor')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].grid(True, alpha=0.3)

axes[1].plot(slippage_df['Slippage Factor'], slippage_df['Annual Return (%)'], 
             marker='o', linewidth=2, color='green')
axes[1].set_title('Annual Return vs Slippage Factor', fontweight='bold')
axes[1].set_xlabel('Slippage Factor')
axes[1].set_ylabel('Annual Return (%)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nSlippage Sensitivity:")
print(slippage_df.to_string(index=False))

## 6. Summary

In [ ]:
print("\n" + "="*80)
print("CAPACITY ANALYSIS SUMMARY")
print("="*80)
print(f"\nTested Capital Range: ${capital_levels[0]:,} - ${capital_levels[-1]:,}")
print(f"Baseline Performance (${capital_levels[0]:,}):")
print(f"  - Sharpe Ratio: {baseline_sharpe:.2f}")
print(f"  - Annual Return: {results_df['Annual Return (%)'].iloc[0]:.2f}%")
print(f"\nPerformance at Maximum Capital (${capital_levels[-1]:,}):")
print(f"  - Sharpe Ratio: {results_df['Sharpe Ratio'].iloc[-1]:.2f}")
print(f"  - Annual Return: {results_df['Annual Return (%)'].iloc[-1]:.2f}%")
print(f"  - Decay: {results_df['Sharpe Decay (%)'].iloc[-1]:.2f}%")
print("\nKey Findings:")
print(f"  - Strategy shows {'minimal' if abs(results_df['Sharpe Decay (%)'].iloc[-1]) < 20 else 'significant'} performance decay")
print(f"  - Slippage impact is {'moderate' if slippage_df['Sharpe Ratio'].iloc[-1] > 1.0 else 'severe'} at high factors")
print("="*80)